# Insper - Supercomputação - Prova Intermediária

Questão sobre GPU / THRUST.


**Questão 4: Normalização de Vetor com Thrust**

Na análise de dados e no processamento de sinais, normalizar um vetor é uma operação comum que ajusta a escala dos elementos para que o vetor tenha uma norma unitária (ou seja, a soma dos quadrados dos elementos é igual a 1). Essa tarefa se torna computacionalmente intensiva para vetores de grande dimensão, fazendo com que a paralelização em GPU seja uma solução atrativa.

**Objetivo**:

Complemente o código abaixo usando a biblioteca Thrust para normalizar um vetor grande. O código inicial cria um vetor com valores aleatórios. Você deve implementar as etapas para:

1. calcular a norma L2 do vetor,
2. dividir cada elemento do vetor por essa norma e
3. imprimir o resultado final.

**Código Pré-Pronto**:

Complete o código com a sua solução.

In [4]:
%%writefile gpu.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/random.h>
#include <iostream>
#include <cmath>
#include <chrono>
#include <cuda_runtime.h>

// Functor para gerar valores aleatórios
struct GerarAleatorios {
    float min, max;

    __host__ __device__
    GerarAleatorios(float _min = 0.0f, float _max = 1.0f) : min(_min), max(_max) {}

    __host__ __device__
    float operator()(unsigned int n) const {
        thrust::default_random_engine engine;
        thrust::uniform_real_distribution<float> distrib(min, max);
        engine.discard(n); // Garante a variação na sequência de números
        return distrib(engine);
    }
};

// Functor para calcular o quadrado dos elementos
struct Quadrado {
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};

// Functor para normalizar o vetor
struct Normalizar {
    float norma;

    __host__ __device__
    Normalizar(float _norma) : norma(_norma) {}

    __host__ __device__
    float operator()(const float& x) const {
        return x / norma;
    }
};

int main() {
    const int N = 1000000;
    thrust::counting_iterator<unsigned int> contador(0);
    thrust::host_vector<float> vetor_host(N);

    // Preenchendo o vetor com valores aleatórios
    thrust::transform(contador, contador + N, vetor_host.begin(), GerarAleatorios(1.f, 2.f));

    // Verificar a memória antes da cópia para GPU
    size_t memoriaLivreAntes, memoriaTotal;
    cudaMemGetInfo(&memoriaLivreAntes, &memoriaTotal);

    // Medir tempo para a cópia do host para a GPU
    auto inicioCopia = std::chrono::high_resolution_clock::now();
    thrust::device_vector<float> vetor_dispositivo = vetor_host;
    auto fimCopia = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duracaoCopia = fimCopia - inicioCopia;

    // Verificar memória após a cópia
    size_t memoriaLivreAposCopia;
    cudaMemGetInfo(&memoriaLivreAposCopia, &memoriaTotal);

    // Calcular a norma L2 do vetor
    auto inicioNorma = std::chrono::high_resolution_clock::now();
    float norma = std::sqrt(thrust::transform_reduce(vetor_dispositivo.begin(), vetor_dispositivo.end(), Quadrado(), 0.0f, thrust::plus<float>()));
    auto fimNorma = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duracaoNorma = fimNorma - inicioNorma;

    std::cout << "Norma calculada: " << norma << std::endl;

    // Normalizar o vetor
    auto inicioNormalizar = std::chrono::high_resolution_clock::now();
    thrust::transform(vetor_dispositivo.begin(), vetor_dispositivo.end(), vetor_dispositivo.begin(), Normalizar(norma));
    auto fimNormalizar = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duracaoNormalizar = fimNormalizar - inicioNormalizar;

    // Imprimir alguns elementos normalizados
    for (int i = 0; i < 40; ++i) {
        std::cout << "Elemento normalizado " << i << ": " << vetor_dispositivo[i] << std::endl;
    }

    // Verificar memória após normalização
    size_t memoriaLivreAposNormalizacao;
    cudaMemGetInfo(&memoriaLivreAposNormalizacao, &memoriaTotal);

    // Imprimir tempos de execução e uso de memória
    std::cout << "\n--- Resultados ---" << std::endl;
    std::cout << "Tempo de cópia para GPU: " << duracaoCopia.count() << " segundos" << std::endl;
    std::cout << "Tempo de cálculo da norma: " << duracaoNorma.count() << " segundos" << std::endl;
    std::cout << "Tempo de normalização: " << duracaoNormalizar.count() << " segundos" << std::endl;
    std::cout << "Memória total da GPU: " << memoriaTotal / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória livre antes da cópia: " << memoriaLivreAntes / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória livre após a cópia: " << memoriaLivreAposCopia / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória livre após a normalização: " << memoriaLivreAposNormalizacao / (1024 * 1024) << " MB" << std::endl;

    return 0;
}

Overwriting gpu.cu


Compilando o código

In [5]:
!nvcc -arch=sm_75 -std=c++14 gpu.cu -o gpu

Exemplo de execução:

In [6]:
%%time
!./gpu

Norma calculada: 1527.27
Elemento normalizado 0: 0.000654778
Elemento normalizado 1: 0.000710439
Elemento normalizado 2: 0.00104851
Elemento normalizado 3: 0.00123856
Elemento normalizado 4: 0.00128854
Elemento normalizado 5: 0.000778965
Elemento normalizado 6: 0.00099195
Elemento normalizado 7: 0.000915364
Elemento normalizado 8: 0.000826904
Elemento normalizado 9: 0.00114159
Elemento normalizado 10: 0.000713396
Elemento normalizado 11: 0.00102169
Elemento normalizado 12: 0.00103599
Elemento normalizado 13: 0.00118484
Elemento normalizado 14: 0.00104233
Elemento normalizado 15: 0.000989813
Elemento normalizado 16: 0.00122875
Elemento normalizado 17: 0.00130631
Elemento normalizado 18: 0.00113026
Elemento normalizado 19: 0.00128766
Elemento normalizado 20: 0.000849295
Elemento normalizado 21: 0.000933725
Elemento normalizado 22: 0.00124372
Elemento normalizado 23: 0.00108232
Elemento normalizado 24: 0.00124505
Elemento normalizado 25: 0.00128434
Elemento normalizado 26: 0.000762611
Ele